In [3]:
# Phase 1
import nltk
from nltk.corpus import brown
import my_toolsv2 as mt
import json

def ex_constitution_corpus():
  themes = {"news":["news", "reviews", "editorial"], 
            "literature":["science_fiction", "romance", "fiction", "mystery"], 
            "sciences":["learned"]}
  nb_instances = 0
  corpus = {}
  for category in themes:
    print(category, ":")
    nb_doc = len(brown.fileids(categories=themes[category]))
    print("  ",nb_doc, "documents")
    nb_instances += nb_doc
    corpus[category] = brown.fileids(categories=themes[category])
  print("NB instances :", nb_instances)
  return corpus

def get_train_test_corpus(corpus):
  import random
  train = {}
  test = {}
  for category, fileids in corpus.items():
    x = int(20*len(fileids)/100)#nb instances pour le train set
    test[category] = []
    print("On prend %s éléments sur %s pour le test"%(str(x),str(len(fileids))))
    for i in range(x):
      id_doc = random.randint(0,len(fileids)-1)#prend un index au hasard
      test[category].append(fileids[id_doc])#stocke le document dans test
      fileids.remove(fileids[id_doc])
    train[category] = fileids #le reste va dans le train set
  dataset = {"train": train, "test":test}
  return dataset

###Corpus complet
corpus = ex_constitution_corpus()

###Séparation train/test
test_train = get_train_test_corpus(corpus)

###Stockage du résultat
test_train_json = json.dumps(test_train, indent=2)
chemin = "train_test.json"
mt.ecrire(test_train_json, chemin)
print("Dataset stocké dans %s"%chemin)

news :
   88 documents
literature :
   88 documents
sciences :
   80 documents
NB instances : 256
On prend 17 éléments sur 88 pour le test
On prend 17 éléments sur 88 pour le test
On prend 16 éléments sur 80 pour le test
Dataset stocké dans train_test.json


In [4]:
# Phase 2
import nltk
from nltk.corpus import brown
import json
import my_toolsv2 as mt

def get_features_light(liste_fichier):
  features_file = {}
  for fileid in liste_fichier:
    features_file[fileid] = {}
    stats_mots = mt.get_stats_occurrence(brown.words(fileid), "usual_verbs.json") # my_toolsv2 ; liste des mots dans chaque fichier, série des mots dans l'ordre
    for feature, valeur in stats_mots.items():
      features_file[fileid][feature] = valeur # couple clé-valeur; utiliser "1/0" ou "input" pour arrêter le programme
  print("->Features extraites:", list(features_file[fileid].keys())[:20],"...")
  return features_file

def get_features(liste):
  features_file = {}
  for fileid in liste:
    features_file[fileid] = {}#on initialise les features du fichier
    # Utilisons notre libraire my_tools pour ajouter des stats sur les mots
    words = brown.words(fileid)
    stats_mots = mt.get_stats_longueur(words)
    for feature, valeur in stats_mots.items():
      features_file[fileid][feature] = valeur
    # Puis sur les phrases
    stats_phrases = mt.get_types_phrases(brown.raw(fileid))
    for feature, valeur in stats_phrases.items():
      features_file[fileid][feature] = valeur
    adverbes = mt.get_effectif_adverbes(words)
    for feature, valeur in adverbes.items():
      features_file[fileid][feature] = valeur
    #... l'entrée varie mais la sortie est un dico {"feature_name":valeur,...}
  print("->Features extraites:", list(features_file[fileid].keys())[:20],"...")
  return features_file

###On récupère la liste des fichiers à traiter
train_test = json.load(open("train_test.json"))
print("\nRécupération de la liste des fichiers")
liste_all_files = mt.get_all_files(train_test)
print("-> %s fichiers"%str(len(liste_all_files))) 

###On extrait les features
print("\nExtraction des features")
features_by_file = get_features_light(liste_all_files)

###On écrit la sortie
print("\nEcriture de la sortie JSON")
filename = "features_by_file.json"
mt.sauvegarder(features_by_file, filename)
print("-> %s"%filename)


Récupération de la liste des fichiers
-> 256 fichiers

Extraction des features
->Features extraites: ['verbs_present_tense', 'verbs_past_tense'] ...

Ecriture de la sortie JSON
-> features_by_file.json


In [9]:
# Phase 3
import nltk
from nltk.corpus import brown
import json
import my_toolsv2 as mt
import os

def get_entete_arff(feature_names, classes):
  """On crée l'entête du arff"""
  lignes_arff = ["@relation TRAIN_DATABASE\n"]#nom de la relation
  for name in feature_names:
    lignes_arff.append("@attribute %s numeric\n"%name)#noms des features
  lignes_arff.append("@attribute classes {%s}\n"%",".join(classes))#les classes
  return lignes_arff

def get_lignes_arff(feature_names, classes, features_by_file):
  """Renvoie la liste des lignes nécessaires pour le fichier ARFF"""
  lignes_arff = get_entete_arff(feature_names, classes)
  lignes_arff.append("@data\n\n")#on passe à la partie DATA
  for nom_classe, l_fichier in classes.items():
    for fileid in l_fichier:#Pour chaque fichier
      l_values = [features_by_file[fileid][name] for name in feature_names]
      l_values.append(nom_classe)#pour l'évaluation
      # La liste des features séparées par des virgules :
      ligne_values = ",".join([str(x) for x in l_values])
      lignes_arff.append(ligne_values)
  return lignes_arff

##pour avoir la répartition train_test
train_test =mt.lire_json("train_test.json")

##la liste des fichiers à traiter
all_fileids = mt.get_all_files(train_test)

###récupération des features précédement extraites
features_by_file = mt.lire_json("features_by_file.json")

### récupération de la liste des noms des features extraites
# arff : dans l'ordre ; 
id_hasard = all_fileids[0]
feature_names = features_by_file[id_hasard].keys() #l'ordre des attributs qui compte

###Définition des combinaisons de features
liste_verbes = mt.lire_json("usual_verbs.json") # jeu de features : quelles sont les caractéristiques qu'on va utiliser
feature_sets = {"all_features" : feature_names,
              "verbs_present_tense" : ["verbs_present_tense"],
               "verbs_past_tense": ["verbs_past_tense"]}

###Création des ARFF
try:
  os.makedirs("arff_files")#création du dossier pour les ranger
  print("Dossier 'arff_files' créé")
except:
  print("Dossier 'arff_files' déjà créé")
  pass

for feature_set_name, feature_list in feature_sets.items(): # Parcourir tous les features
  print("\nFeature set : %s"%feature_set_name)
  for dataset, classes in train_test.items():
    print("  Processing %s set"%dataset)
    filename = "arff_files/%s__%s.arff"%(feature_set_name, dataset)
    lignes_arff = get_lignes_arff(feature_list, classes, features_by_file)
    mt.ecrire("\n".join(lignes_arff), filename)
    print("    ->sortie = %s"%filename)

Dossier 'arff_files' créé

Feature set : all_features
  Processing train set
    ->sortie = arff_files/all_features__train.arff
  Processing test set
    ->sortie = arff_files/all_features__test.arff

Feature set : verbs_present_tense
  Processing train set
    ->sortie = arff_files/verbs_present_tense__train.arff
  Processing test set
    ->sortie = arff_files/verbs_present_tense__test.arff

Feature set : verbs_past_tense
  Processing train set
    ->sortie = arff_files/verbs_past_tense__train.arff
  Processing test set
    ->sortie = arff_files/verbs_past_tense__test.arff
